In [10]:
!unzip -o mysql-connector-java-8.0.27.zip


Archive:  mysql-connector-java-8.0.27.zip
  inflating: mysql-connector-java-8.0.27/CHANGES  
  inflating: mysql-connector-java-8.0.27/INFO_BIN  
  inflating: mysql-connector-java-8.0.27/INFO_SRC  
  inflating: mysql-connector-java-8.0.27/LICENSE  
  inflating: mysql-connector-java-8.0.27/README  
  inflating: mysql-connector-java-8.0.27/build.xml  
  inflating: mysql-connector-java-8.0.27/mysql-connector-java-8.0.27.jar  
  inflating: mysql-connector-java-8.0.27/src/build/java/documentation/ErrorMappingsDocGenerator.java  
  inflating: mysql-connector-java-8.0.27/src/build/java/documentation/PropertiesDocGenerator.java  
  inflating: mysql-connector-java-8.0.27/src/build/java/instrumentation/AddMethods.java  
  inflating: mysql-connector-java-8.0.27/src/build/java/instrumentation/CommonChecks.java  
  inflating: mysql-connector-java-8.0.27/src/build/java/instrumentation/TranslateExceptions.java  
  inflating: mysql-connector-java-8.0.27/src/build/misc/Product.wxs  
  inflating: mysql-c

In [1]:
from pyspark.sql import SparkSession

# Path to the MySQL JDBC driver jar file
mysql_jar_path = './mysql-connector-java-8.0.27/mysql-connector-java-8.0.27.jar'  # Update this path

# Initialize Spark session
mongo_connector_package = "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1"

# Initialize Spark session
spark = SparkSession.builder \
    .appName("StockProcessing") \
    .config("spark.jars", mysql_jar_path) \
    .config("spark.driver.extraClassPath", mysql_jar_path) \
    .config("spark.executor.extraClassPath", mysql_jar_path) \
    .config("spark.mongodb.input.uri", "mongodb://127.0.0.1/Stock_DB.stock_prices") \
    .config("spark.mongodb.output.uri", "mongodb://127.0.0.1/Stock_DB.processed_stock_prices") \
    .config("spark.jars.packages", mongo_connector_package) \
    .config("spark.driver.extraJavaOptions", "-Djava.net.preferIPv4Stack=true") \
    .getOrCreate()

24/11/10 20:40:35 WARN Utils: Your hostname, yousafzai resolves to a loopback address: 127.0.1.1; using 192.168.100.98 instead (on interface wlp2s0)
24/11/10 20:40:35 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/yousafzai/.local/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/yousafzai/.ivy2/cache
The jars for the packages stored in: /home/yousafzai/.ivy2/jars
org.mongodb.spark#mongo-spark-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-f85f1c42-17aa-4832-bbf5-8c88644b6ec7;1.0
	confs: [default]
	found org.mongodb.spark#mongo-spark-connector_2.12;3.0.1 in central
	found org.mongodb#mongodb-driver-sync;4.0.5 in central
	found org.mongodb#bson;4.0.5 in central
	found org.mongodb#mongodb-driver-core;4.0.5 in central
downloading https://repo1.maven.org/maven2/org/mongodb/spark/mongo-spark-connector_2.12/3.0.1/mongo-spark-connector_2.12-3.0.1.jar ...
	[SUCCESSFUL ] org.mongodb.spark#mongo-spark-connector_2.12;3.0.1!mongo-spark-connector_2.12.jar (6455ms)
downloading https://repo1.maven.org/maven2/org/mongodb/mongodb-driver-sync/4.0.5/mongodb-driver-sync-4.0.5.jar ...
	[SUCCESSFUL ] org.mongodb#mongodb-driver-sync;4.0.5!mongodb-driver-sync.jar (901ms)
downloading https://repo1.m

In [2]:
# Load MySQL data
stock_df = spark.read.format("jdbc") \
    .option("url", "jdbc:mysql://localhost:3306/Stock_DB") \
    .option("dbtable", "stock_prices") \
    .option("user", "root") \
    .option("password", "Root@1234") \
    .load()


In [3]:
avg_prices = stock_df.groupBy("ticker", "date").avg("close").withColumnRenamed("avg(close)", "avg_close")

# Save processed data back to MongoDB
avg_prices.write.format("mongo").mode("overwrite").save()

# Database Comparison

In [9]:
import time

start_time = time.time()
# Insert data to MySQL or MongoDB
print("Insertion time:", time.time() - start_time)


Insertion time: 0.00021266937255859375


In [11]:
import mysql.connector
import time
from pymongo import MongoClient

# MySQL connection setup
mysql_conn = mysql.connector.connect(
    host="localhost",       # Replace with your MySQL server's address
    user="root",   # Replace with your MySQL username
    password="Root@1234",  # Replace with your MySQL password
    database="Stock_DB"     # Replace with your MySQL database name
)

mysql_cursor = mysql_conn.cursor()

# MongoDB connection setup
mongo_client = MongoClient("mongodb://localhost:27017/")
mongo_db = mongo_client["Stock_DB"]

# Timing MySQL query
start_time = time.time()
mysql_cursor.execute("SELECT * FROM stock_prices WHERE ticker='AAPL'")
mysql_results = mysql_cursor.fetchall()
print("MySQL query time:", time.time() - start_time)

# Timing MongoDB query
start_time = time.time()
mongo_results = list(mongo_db.stock_prices.find({"ticker": "AAPL"}))
print("MongoDB query time:", time.time() - start_time)

# Closing connections
mysql_cursor.close()
mysql_conn.close()
mongo_client.close()


MySQL query time: 0.012931108474731445
MongoDB query time: 0.20046067237854004
